# Quick Visualizer SPHinXsys - Water/Fluid interactions

# Run the simulation
This part is correct as of 21/07/2025, but in the future some files might be named different. Alter if needed.

In [3]:
import shutil 
#you might have to rename it if your system gives it a different name
shutil.move("../../lib/test_2d_dambreak_python.cpython-312-x86_64-linux-gnu.so", "test_2d_dambreak_python.cpython-312-x86_64-linux-gnu.so")
shutil.move("../input", "input")

'test_2d_dambreak_python.cpython-312-x86_64-linux-gnu.so'

In [1]:
%run pybind_test.py

ModuleNotFoundError: No module named 'test_2d_dambreak_python'

In [ ]:
# %pip install pyvista vtk

In [ ]:
import glob
import vtk
import numpy as np
import pyvista as pv
import matplotlib.pyplot as plt
import time

## Import the required files

Make sure you have run the SPHinxSys programm and created the required the .vtp files. Change the directory to what corresponds to your .vtp files (usually ends in 'bin/output/WaterBody____.vtp', but you need to check yourself)

In [ ]:
water     = glob.glob("output/WaterBody_*.vtp")
frames    = len(water)
wall      = glob.glob("output/Wall*.vtp")

## Read the required files and generate the grid

Read the required files and make an initial plot, this is just to test whether your code reads the program correctly. More documentation can be found here: https://examples.vtk.org/site/Python/IO/ReadPolyData/ , https://vtk.org/doc/nightly/html/classvtkPoints.html and https://examples.vtk.org/site/Python/Snippets/ReadPolyData/ .

##### 'location' and 'speed' are lists
##### 'grid' and 'velocity' are arrays
Currently the code outputs arrays, but if you prefer lists for other uses, know that you need to call location and speed.

In [ ]:
reader   = vtk.vtkXMLPolyDataReader()

In [ ]:
def reading(frame):    #This function reads a certain frame and generates the output
    reader.SetFileName(water[frame])
    reader.Update() 
    
    output      = reader.GetOutput()
    points      = output.GetPoints()
    n_points    = output.GetNumberOfPoints()
    speed_array = output.GetPointData().GetArray("Velocity")
    location    = []
    speed       = []
    for i in range(int(n_points)):
        current_point    = points.GetPoint(i)
        current_speed    = speed_array.GetTuple3(i)
        current_speed    = np.linalg.norm(current_speed)
        location.append(current_point)
        speed.append(current_speed)
    grid        = np.stack(location)
    velocity    = np.stack(speed   )
    return grid, velocity

In [ ]:
def reading_wall(wall):  #This function reads the wall boundary
    reader.SetFileName(wall[0])
    reader.Update() 
    
    output     = reader.GetOutput()
    points     = output.GetPoints()
    n_points   = output.GetNumberOfPoints()
    barrier    = []
    for i in range(n_points):
        barrier.append(points.GetPoint(i))
    grid       = np.stack(barrier)
    return grid

## Make a .gif of your points.

https://docs.pyvista.org/examples/02-plot/gif.html has a great example of how .gif's are created in PyVista. You can change some things according to your preference. Like the particle size, opacity, but you can also adjust the colormap to your preferences. Use https://matplotlib.org/stable/gallery/color/colormap_reference.html where you can see all the colormaps that are included in Matplotlib.

You can change the camera position in many ways, but you can also manually place the camera in a position. Please refer to: https://docs.pyvista.org/api/plotting/_autosummary/pyvista.plotter.camera_position

##### note: 
It is a known problem that the colorbar is not extremely stable in PyVista. As soon as I've found a solution for that I'll implement it


In [ ]:
#create the plot itself
plotter = pv.Plotter(notebook=False, off_screen=True)

#This is the wall boundary
#(don't delete the section, it helps the camera position)
plotter.add_points(
    reading_wall(wall),
    opacity= 1.0,                   #set this to zero if you don't want to see the wall at all 
    render_points_as_spheres=True,
    point_size=5                      #change the size of the particles accordingly
)

#These are the initial fluid particles
plotter.add_points(
    reading(0)[0],
    scalars = reading(0)[1],
    opacity= 1,                       #opacity of the water particles. If you want to have a cut through a certain axis, see the next kernel
    cmap='turbo',    
    clim=[0, 1],                    
    render_points_as_spheres=True,
    point_size=5                      #change the size of the particles accordingly
)
plotter.set_background("white")
plotter.camera_position = 'xy'        #the first letter is the horizontal axis the second axis is the vertical axis. 

#Open the give and give it a name
plotter.open_gif("test_2d_dambreak.gif")   

nframe = 15
print(f"start creating .gif")

starttime = time.time()
midtime   = starttime
for j in range(frames):
    pts, vel  = reading(j)
    plotter.update_coordinates(pts, render=False)
    plotter.update_scalars(vel, render=False)
    plotter.write_frame()
    if j % int(frames/10+1) == 0:
        print(f"Currently at {int(j/frames*10)/10}% of {int(frames)} frames. Total time {int(time.time() - starttime)} seconds, time since last update {int(time.time() - midtime)} seconds ")
        midtime = time.time()

print(f"All done compiling frames")

plotter.close()
print(f".gif created, enjoy!")